In [1]:
from collections import Counter
import json
import urllib2
import sys

import pandas as pd
from ggplot import *


counter = Counter()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
group_name = "DMIWinter19AlexAmplify"

#static endpoint from https://github.com/tracking-exposed/facebook/wiki/Researchers-API-and-how-to-download-the-data
api_end = 'https://facebook.tracking.exposed/api/v1/research/data/l:'


print("Getting data from\n" + api_end + group_name)

data = urllib2.urlopen(api_end + group_name).read()
print(data)

fname = group_name + '.json'

f = open(fname, 'wb+')
f.write(data)
f.close()

In [2]:
f = open(group_name.strip() + ".json", 'rb')
posts = json.loads(f.read())
f.close()

fh = open(group_name.strip() + 'links.csv', 'wb+')
fh.write(b"Publisher \; Capture\; Links")
for post in posts["results"]:
    links = []
    imp_tme = b""
    link = ""
    publish = post["attributions"][0]["content"]

    if "impressionTime" in post:
        imp_time = post["impressionTime"]

    if "externalLinks" in post:
        links = [external['link'] for external in post["externalLinks"]]

    if len(links) > 0:
        link = ','.join(links)
        for l in links:
            counter[l] += 1
    
    write_data = ';'.join([publish, imp_time, link])

    fh.write(write_data.encode('utf-8') + b"\n")

fh.close()

NameError: name 'counter' is not defined

In [ ]:
for link_item in counter.most_common(5):
    print("Count {0} \t Link {1}".format(link_item[1], link_item[0]))

Code to identify the person, publisher and time of a link being posted. The aim is to see if the same link appears from different publishers and times suggesting a push of content.

A todo would be to query the url shorteners to get the full name from the redirect to see if URLs are being masked behind aliases. 

The results, captured in DMIWinter19AlexAmplifyperson.csv are fairly innocuous here. 

In [ ]:
link_name = "https://bzfd.it/2CUoJk6"

f = open(group_name.strip() + ".json", 'rb')
posts = json.loads(f.read())
f.close()

fh = open(group_name.strip() + 'person.csv', 'wb+')
fh.write(b"Publisher,Capture,UserId\n")
for post in posts["results"]:
    imp_time = b"No Date"

    if "externalLinks" in post:
        for l in post["externalLinks"]:
            if link_name == l['link']:
                if "impressionTime" in post:
                    imp_time = post["impressionTime"]
                userid = str(post["userId"])
                write_data = ','.join([post["attributions"][0]["content"], imp_time, userid])
                fh.write(write_data.encode('utf-8') + b"\n")

fh.close()

In [ ]:
link_df = pd.read_csv(group_name.strip() + 'person.csv')

In [ ]:
ggplot(aes(x="Capture", y="Publisher", weight="Publisher"), data=link_df) + geom_point()